In [1]:
#-*- coding:utf-8 -*-
from commonTool import *


running on Windows


In [ ]:
# file: dailyPrice.py

# KRX에서 지정한 날짜의 주가를 크롤링해 날짜별로 저장
# beginDay[, endDay] 지정 가능. 없으면 조회 날짜의 주가 크롤링

def writeDailyPrice(dayNo):
    outputDirPath = outputRawPath + 'priceDaily' + os.path.sep + dayNo[0:4] + os.path.sep
    mkdir(outputDirPath)

    print('Day', dayNo, 'Output Folder:', outputDirPath, flush=True)
    writeDailyPriceFromKRX(dayNo, outputDirPath)
    

jobName = 'get daily price'
begin(jobName)

d = dt.datetime.today()

beginDay = d.strftime('%Y%m%d')
if len(sys.argv) >= 2 and len(sys.argv[1]) == 8:
    beginDay = sys.argv[1]
    d = dt.datetime.strptime(beginDay, '%Y%m%d').date()
    
endDay = beginDay
if len(sys.argv) >= 3 and len(sys.argv[2]) == 8:
    endDay = sys.argv[2]

print('range', beginDay, endDay)

while beginDay <= endDay:
    writeDailyPrice(beginDay)
    d = d + dt.timedelta(days=1)
    beginDay = d.strftime('%Y%m%d')

end(jobName)


In [ ]:
# file: adjPriceYear.py
# Naver에서 수정주가를 년단위로 조회하여 저장

def writeAdjustPrice(codes, year, outFile):
    noLine = 0
    for code in codes:
        sTick = tm.time()
        noLine += 1
        print(noLine, code,'processing', flush=True)
        
        beginDay = year + '0101'
        endDay = year + '1231'

        # resp = fetchMonthStockData(code, beginDay, endDay)
        resp = fetchDailyStockData(code, beginDay, endDay)

        writeStockData(code, outFile, resp)

        print(noLine, code,'done', tm.time() - sTick, flush=True)


jobName = 'get adjust price'
begin(jobName)

(codes, types, referCodes) = getCompanyCodeFromFile(codePathFile)
print('codes', len(codes), len(types), len(referCodes)) # 첫 번째, 두 번째 값 같아야 함.


d = dt.datetime.today()
year = d.strftime('%Y')

if len(sys.argv) >= 2 and len(sys.argv[1]) == 4:
    year = sys.argv[1]

print('Fetching Year', year, flush=True)


outputDirPath = outputRawPath + 'adjustPrice' + os.path.sep
mkdir(outputDirPath)

outputDirPath += 'adjDailyPrice-' + year + '.csv'

outFile = open(outputDirPath, 'w', encoding='utf-8')
outFile.write('종목코드, 날짜, 시가, 고가, 저가, 종가, 거래량, 외국인소진율\n');

writeAdjustPrice(codes, year, outFile)

outFile.close()

end(jobName)
    

In [ ]:
# file: uploadPrice.py [year]

jobName = 'upload price info to db'
begin(jobName)

year = today()[0:4]

if len(sys.argv) >= 2 and len(sys.argv[1]) == 4:
    year = sys.argv[1]

scriptPathName = crawlegoScriptPath + 'UPLOAD-PRICE.xml'

parameter = {
    'YEAR': year,
    'ADJ_PATH': outputRawPath + 'adjustPrice' + os.path.sep + 'adjDailyPrice',
    'KRX_PATH': outputRawPath + 'priceDaily' + os.path.sep + year + os.path.sep,
    'DO_SERVER': '13.124.29.70'
}

retCode = runDashScript(scriptPathName, parameter)

end(jobName)

In [ ]:
# SP-1. 워킹 데이만 뽑아 저장하는 스크립트 실행
workingDayPath = 'D:\\work\\jupyter\\krx\output\\workingDay\\'

scriptPathName = crawlegoScriptPath + 'ZTMP_DATE.xml'
for year in range(2012, 2022):
    parameter = {
        'YEAR': str(year),
        'OUT_PATH': workingDayPath,
        'DO_SERVER': '13.124.29.70'
    }

    print(year, 'ret code', runDashScript(scriptPathName, parameter))

In [3]:
# SP-2. 워킹 데이 중 달별로 기준일 추출하는 스크립트 실행
# 매달 첫 째날, 둘 째날, 마지막 날, 마지막 전날 총 4일을 저장함
# resourceDir에 BASIS_DAYs.txt와 주가 쿼리를 위한 날짜 조건을 담고 있는 BASIS-CONDITION.txt 파일 생성

scriptPathName = crawlegoScriptPath + 'ZTMP_DATE_MINMAX.xml'
parameter = {
    'IN_PATH': workingDayPath,
    'OUT_PATH': resourceDir
}

print('ret code', runDashScript(scriptPathName, parameter))

java -Dfile.encoding=utf8 -Duser.timezone=GMT -jar D:\work\jupyter\krx\jar\crawlego-1.0.0.jar D:\work\jupyter\krx\script\ZTMP_DATE_MINMAX.xml IN_PATH=D:\work\jupyter\krx\output\workingDay\ OUT_PATH=D:\work\jupyter\krx\resource\
ret code 0


In [6]:
# SP-3. SP-2의 BASIS-CONDITION.txt 파일을 읽어 해당 날짜의 주가를 쿼리하여 임시폴더에 저장함.

scriptPathName = crawlegoScriptPath + 'ZTMP_STOCK_PRICE.xml'

bf = open(resourceDir + 'BASIS-CONDITION.txt', 'r', encoding='utf-8')

lineText = bf.readline() # 제목

while True:
    lineText = bf.readline()
    if not lineText:
        break
        
    lineText = lineText.strip()
    if len(lineText) <= 0:
        continue
        
    dayData = lineText.split('\t')
    
    # 금년만 계산하려면 아래 라인 살리기. 이전 년도는 BASIS가 바뀌지 않는한 변함 없음
    if dayData[0] != today()[0:4]: continue
    
    parameter = {
        'YEAR': dayData[0],
        'OUT_PATH': temporaryPath,
        'COND': dayData[1],
        'DO_SERVER': '13.124.29.70'
    }

    print(dayData[0], 'ret code', runDashScript(scriptPathName, parameter))

bf.close()

java -Dfile.encoding=utf8 -Duser.timezone=GMT -jar D:\work\jupyter\krx\jar\crawlego-1.0.0.jar D:\work\jupyter\krx\script\ZTMP_STOCK_PRICE.xml YEAR=2021 OUT_PATH=D:\work\jupyter\krx\intermediate\ COND='20210104','20210105','20210128','20210129','20210201','20210202','20210225','20210226','20210302','20210303','20210330','20210331','20210401','20210402','20210429','20210430','20210503','20210504','20210528','20210531','20210601','20210602','20210617','20210618' DO_SERVER=13.124.29.70
2021 ret code 0


In [8]:
# SP-4. 기준이 되는 날짜의 주가를 하나로 합쳐 DB에 업로딩.

scriptPathName = crawlegoScriptPath + 'ZTMP_MERGE_PRICE.xml'

parameter = {
    'IN_PATH': temporaryPath,
    'DO_SERVER': '13.124.29.70'
}

print('ret code', runDashScript(scriptPathName, parameter))

java -Dfile.encoding=utf8 -Duser.timezone=GMT -jar D:\work\jupyter\krx\jar\crawlego-1.0.0.jar D:\work\jupyter\krx\script\ZTMP_MERGE_PRICE.xml IN_PATH=D:\work\jupyter\krx\intermediate\ DO_SERVER=13.124.29.70
ret code 1
